In [1]:
cd /Users/hazelchui/Desktop/UChicago/Perspectives

/Users/hazelchui/Desktop/UChicago/Perspectives


In [2]:
import csv
import sys
import re
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import numpy as np

In [18]:
# Import the list of winners and nominees  
oscars_lst = pd.read_csv("/Users/hazelchui/Desktop/UChicago/Perspectives/final_oscars_list.csv")
oscars_lst = oscars_lst.fillna(0)
oscars_lst['release_year'] = (oscars_lst['year']-1).astype(str)
oscars_lst

,year,movie,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year
0,2015,Birdman or (The Unexpected Virtue of Ignorance...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
1,2015,"American Sniper – Clint Eastwood, Robert Loren...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
2,2015,Boyhood – Richard Linklater and Cathleen Suthe...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
3,2015,"The Grand Budapest Hotel – Wes Anderson, Scott...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
4,2015,"The Imitation Game – Nora Grossman, Ido Ostrow...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,2008,Peter & the Wolf – Suzie Templeton and Hugh We...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2007
485,2008,Even Pigeons Go to Heaven (Même les pigeons vo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007
486,2008,I Met the Walrus – Josh Raskin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007
487,2008,Madame Tutli-Putli – Chris Lavis and Maciek Sz...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007


In [21]:
# Clean movie name
oscars_lst['movie'] = oscars_lst['movie'].str.extract(r'(.*?)\xa0–')
oscars_lst
# oscars_lst['movie'] = oscars_lst['movie'].str.replace(r'\xa0', ' ')

oscars_lst.rename(columns={'movie':'movie_name'}, inplace = True)

In [22]:
# 474 movies
oscars_lst = oscars_lst.dropna(subset=['movie_name'])
oscars_lst

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
1,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
2,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
3,2015,The Grand Budapest Hotel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
4,2015,The Imitation Game,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,2008,Peter & the Wolf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2007
485,2008,Even Pigeons Go to Heaven (Même les pigeons vo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007
486,2008,I Met the Walrus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007
487,2008,Madame Tutli-Putli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007


In [193]:
#oscars_lst.loc[(oscars_lst['movie_name'].isna())]

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year


In [195]:
release_year = list(oscars_lst['release_year'])
movie = list(oscars_lst['movie_name'])
movie_lst = list(zip(release_year, movie))

In [196]:
movie_lst
for idx, (release_year, movie_name) in enumerate(movie_lst):
    print(idx, movie_name)

0 Birdman or (The Unexpected Virtue of Ignorance)
1 American Sniper
2 Boyhood
3 The Grand Budapest Hotel
4 The Imitation Game
5 Selma
6 The Theory of Everything
7 Whiplash
8 Big Hero 6
9 The Boxtrolls
10 How to Train Your Dragon 2
11 Song of the Sea
12 The Tale of the Princess Kaguya
13 Ida (Poland) in Polish
14 Leviathan (Russia) in Russian
15 Tangerines (Estonia) in Estonian and Russian
16 Timbuktu (Mauritania) in French 
17 Wild Tales (Argentina) in Spanish 
18 Citizenfour
19 Finding Vivian Maier
20 Last Days in Vietnam
21 The Salt of the Earth
22 Virunga
23 Crisis Hotline: Veterans Press 1
24 Joanna
25 Our Curse
26 The Reaper
27 White Earth
28 The Phone Call
29 Aya
30 Boogaloo and Graham
31 Butter Lamp
32 Parvaneh
33 Feast
34 The Bigger Picture
35 The Dam Keeper
36 Me and My Moulton
37 A Single Life
38 Spotlight
39 The Big Short
40 Bridge of Spies
41 Brooklyn
42 Mad Max: Fury Road
43 The Martian
44 The Revenant 
45 Room
46 Inside Out
47 Anomalisa
48 Boy and the World
49 Shaun the S

In [198]:
len(movie_lst)

474

In [199]:
# Grab the imdb link for each movie
def get_imdb_links(movie_lst):     
    
    driver = webdriver.Safari()
    driver.get("https://www.google.com/")
    m = driver.find_element_by_name("q")
    link_lst = []
    for idx, (release_year, movie_name) in enumerate(movie_lst):
        time.sleep(1)
        driver.get("https://www.google.com/")
        m = driver.find_element_by_name("q")
        m.send_keys(movie_name + " IMDB " + release_year)
        m.send_keys(Keys.ENTER)
        time.sleep(2)
        elems = driver.find_elements_by_css_selector(".yuRUbf [href]")
        link = [elem.get_attribute('href') for elem in elems][0]
        if link.startswith('https://www.imdb.com/title/'):
            link_lst.append((release_year, movie_name, link))
            print(idx, movie_name)
        else:
            print('Not on IMDB')
    driver.close()
    return link_lst

In [200]:
results = get_imdb_links([i for i in movie_lst[192:]])

/var/folders/nk/mc55cdxs3h39bqv94923ds1m0000gn/T/ipykernel_822/1554982344.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  m = driver.find_element_by_name("q")
/var/folders/nk/mc55cdxs3h39bqv94923ds1m0000gn/T/ipykernel_822/1554982344.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  m = driver.find_element_by_name("q")
/var/folders/nk/mc55cdxs3h39bqv94923ds1m0000gn/T/ipykernel_822/1554982344.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elems = driver.find_elements_by_css_selector(".yuRUbf [href]")


0 Parasite
Not on IMDB
2 The Irishman
3 Jojo Rabbit
4 Joker
5 Little Women
6 Marriage Story
7 1917
8 Once Upon a Time in Hollywood
9 Toy Story 4
10 How to Train Your Dragon: The Hidden World
11 I Lost My Body
12 Klaus
13 Missing Link
14 Parasite (South Korea) in Korean
15 Corpus Christi (Poland) in Polish
16 Honeyland (North Macedonia) in Turkish and Macedonian[32]
17 Les Misérables (France) in French
18 Pain and Glory (Spain) in Spanish
19 American Factory
20 The Cave
21 The Edge of Democracy
22 For Sama
23 Honeyland
24 Learning to Skateboard in a Warzone (If You're a Girl)
25 In the Absence
26 Life Overtakes Me
27 St. Louis Superman
28 Walk Run Cha-Cha
29 The Neighbors' Window
30 Brotherhood
31 Nefta Football Club
32 Saria
33 A Sister
34 Hair Love
35 Dcera (Daughter)
36 Kitbull
37 Mémorable
38 Sister
39 12 Years a Slave
40 American Hustle
41 Captain Phillips
42 Dallas Buyers Club
43 Gravity
44 Her
45 Nebraska
46 Philomena
47 The Wolf of Wall Street
48 Frozen
49 The Croods
50 Despicab

In [205]:
results

[('2019', 'Parasite', 'https://www.imdb.com/title/tt6751668/'),
 ('2019', 'The Irishman', 'https://www.imdb.com/title/tt1302006/'),
 ('2019', 'Jojo Rabbit', 'https://www.imdb.com/title/tt2584384/'),
 ('2019', 'Joker', 'https://www.imdb.com/title/tt7286456/'),
 ('2019', 'Little Women', 'https://www.imdb.com/title/tt3281548/'),
 ('2019', 'Marriage Story', 'https://www.imdb.com/title/tt7653254/'),
 ('2019', '1917', 'https://www.imdb.com/title/tt8579674/'),
 ('2019',
  'Once Upon a Time in Hollywood',
  'https://www.imdb.com/title/tt7131622/'),
 ('2019', 'Toy Story 4', 'https://www.imdb.com/title/tt1979376/'),
 ('2019',
  'How to Train Your Dragon: The Hidden World',
  'https://www.imdb.com/title/tt2386490/'),
 ('2019', 'I Lost My Body', 'https://www.imdb.com/title/tt9806192/'),
 ('2019', 'Klaus', 'https://www.imdb.com/title/tt4729430/'),
 ('2019', 'Missing Link', 'https://www.imdb.com/title/tt6348138/'),
 ('2019',
  'Parasite\xa0(South Korea) in\xa0Korean',
  'https://www.imdb.com/title/t

In [234]:
# Scrape other info
# Need to rescrape
def get_other_info(results):
    
    csv_file = open('info.csv', 'a', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['release_year', 'movie_name', 'link', 
                         'oscar_num', 'opening_box', 'opening_date']) 
    driver = webdriver.Safari()
    for idx, (release_year, movie_name, link) in enumerate(results):
        driver.get(link)
        time.sleep(3)
        
        # No. of other oscars 
        oscar_num = re.findall(r'Won (.*?) Oscars',  driver.find_element(By.CSS_SELECTOR, "#__next>main >div>section>div>section>div>div>section>div>ul>li>a").text)
        if oscar_num != []:
            oscar_num = int(oscar_num[0])
        else:
            oscar_num = 0

        # Opening Box office & opening date 
        opening_weekend = driver.find_elements(By.CSS_SELECTOR, "#__next > main > div > section > div > section > div > div > section > div > ul > li > div > ul > li > span")
        op = []
        for i in opening_weekend:
            op.append(i.text)
        for index, mo in enumerate(op):
            opening_date = re.findall(r'[A-Z][a-z][a-z]\s[0-9]+[,]\s[0-9]+', mo)
            if opening_date != []:
                opening_d = opening_date[0] 
                opening_box = op[index-1]
                csv_writer.writerow([release_year, movie_name, link, 
                         oscar_num, opening_box, opening_d])
                break
            elif index == len(op)-1:
                opening_d = 'NA'
                opening_box = 'NA'
                csv_writer.writerow([release_year, movie_name, link, 
                 oscar_num, opening_box, opening_d])
        print(idx, movie_name)
        
    driver.close()                                    
    

In [167]:
get_other_info(results)

In [206]:
# Scrape the reviews
def get_reviews(links):
    csv_file = open('reviews.csv', 'a', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['release_year', 'movie_id','movie_name', 'date', 'user_name', 
                         'review', 'rating']) 
    for idx, (release_year, movie_name, link) in enumerate(links): 
        start_url = link + 'reviews?ref_=tt_urv'
        link = link + 'reviews/_ajax'

        params = {
            'ref_': 'undefined',
            'paginationKey': ''
        }

        with requests.Session() as s:
            s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
            res = s.get(start_url)
            while True:
                soup = BeautifulSoup(res.text,"lxml")
                for item in soup.select(".review-container"):
                    try:
                        rating_fraction = item.select_one("span.rating-other-user-rating").get_text(strip=True)
                        rating = re.findall(r'(.*?)/10', rating_fraction)[0]
                    except:
                        rating = 'NA'
                    movie_id = re.findall(r'https://www.imdb.com/title/(.*?)/', link)[0]
                    reviewer_name = item.select_one("span.display-name-link > a").get_text(strip=True)
                    date = item.select_one("span.review-date").get_text(strip=True)
                    review = item.select_one(".content > .text").get_text(strip=True)
                    csv_writer.writerow([release_year, movie_id, movie_name, date, reviewer_name,
                             review, rating])
                
                try:
                    pagination_key = soup.select_one(".load-more-data[data-key]").get("data-key")
                except AttributeError:
                    break
                params['paginationKey'] = pagination_key
                res = s.get(link, params=params)
        print(idx, movie_name)

In [211]:
get_reviews(results[188:])
reviews = pd.read_csv('reviews.csv')
reviews.tail()

0 The Blind Side 
1 District 9 
2 An Education 
3 Inglourious Basterds 
4 Precious: Based on the Novel 'Push' by Sapphire 
5 A Serious Man 
6 Up 
7 Up in the Air 
8 Up
9 Coraline
10 Fantastic Mr. Fox
11 The Princess and the Frog
12 The Secret of Kells
13 The Secret in Their Eyes (Argentina) in Spanish
14 Ajami (Israel) in Arabic and Hebrew
15 The Milk of Sorrow (Peru) in Spanish and Quechua
16 A Prophet (France) in French, Corsican and Arabic
17 The White Ribbon (Germany) in German
18 The Cove
19 Burma VJ
20 Food, Inc.
21 The Most Dangerous Man in America: Daniel Ellsberg and the Pentagon Papers
22 Which Way Home
23 China's Unnatural Disaster: The Tears of Sichuan Province
24 The Last Campaign of Governor Booth Gardner
25 The Last Truck: Closing of a GM Plant
26 Rabbit à la Berlin
27 The New Tenants
28 The Door
29 Instead of Abracadabra
30 Kavi
31 Miracle Fish
32 Logorama
33 French Roast
34 Granny O'Grimm's Sleeping Beauty
35 The Lady and the Reaper
36 A Matter of Loaf and Death
37 Slu

,release_year,movie_id,movie_name,date,user_name,review,rating
185784,2007,tt0874952,My Love (Moya Lyubov),6 September 2014,Vincentiu,impressionist art. delicate theme. Alexandr Pe...,NaN
185785,2007,tt0874952,My Love (Moya Lyubov),18 May 2017,Kirpianuscus,it is a great travel. across a feeling. across...,NaN
185786,2007,tt0874952,My Love (Moya Lyubov),17 July 2015,Horst_In_Translation,This is a Russian animated short film that run...,4
185787,2007,tt0874952,My Love (Moya Lyubov),21 June 2018,Jithindurden,The love and lust of an adolescent boy. The in...,9
185788,2007,tt0874952,My Love (Moya Lyubov),9 August 2021,benoyum111,Where the story is only may be interesting for...,8
